DB Smoke Test: During the initial setup, a check was performed to see whether database objects (schema/tables) could be created. 

The creation of a schema (CREATE SCHEMA) failed with permission denied, indicating restricted user rights in the database. In order to be able to work reproducibly, the smoke test was adapted: First, the pure connection was validated via SELECT, then it was checked whether the target schema existed and whether tables could be created there. Based on this rights check, a decision is made as to whether to work in the intended schema or alternatively in the public schema.



In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # lädt Variablen aus .env

PG_USER = os.getenv("POSTGRES_USER")
PG_PASS = os.getenv("POSTGRES_PASS")
PG_HOST = os.getenv("POSTGRES_HOST")
PG_PORT = os.getenv("POSTGRES_PORT", "5432")
PG_DB   = os.getenv("POSTGRES_DB")
PG_SCHEMA = os.getenv("POSTGRES_SCHEMA", "public")

missing = [k for k,v in {
    "POSTGRES_USER": PG_USER,
    "POSTGRES_PASS": PG_PASS,
    "POSTGRES_HOST": PG_HOST,
    "POSTGRES_DB": PG_DB,
    "POSTGRES_SCHEMA": PG_SCHEMA
}.items() if not v]

if missing:
    raise ValueError(f"Missing env vars in .env: {missing}")

print("Env loaded OK. Schema:", PG_SCHEMA)


Env loaded OK. Schema: s_patrickpaubandt


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

print("USER:", os.getenv("POSTGRES_USER"))
print("HOST:", os.getenv("POSTGRES_HOST"))
print("PORT:", os.getenv("POSTGRES_PORT"))
print("DB  :", os.getenv("POSTGRES_DB"))
print("SCHEMA:", os.getenv("POSTGRES_SCHEMA"))
print("PASS length:", len(os.getenv("POSTGRES_PASS") or ""))
print("PASS has spaces:", " " in (os.getenv("POSTGRES_PASS") or ""))


USER: patrickpaubandt
HOST: data-analytics-course-2.c8g8r1deus2v.eu-central-1.rds.amazonaws.com
PORT: 5432
DB  : nf_da_onl_13102025
SCHEMA: s_patrickpaubandt
PASS length: 16
PASS has spaces: False


In [4]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import os

PG_USER = os.getenv("POSTGRES_USER")
PG_PASS = quote_plus(os.getenv("POSTGRES_PASS"))
PG_HOST = os.getenv("POSTGRES_HOST")
PG_PORT = os.getenv("POSTGRES_PORT", "5432")
PG_DB   = os.getenv("POSTGRES_DB")
PG_SCHEMA = os.getenv("POSTGRES_SCHEMA", "public")

url = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine = create_engine(url, future=True)

with engine.connect() as conn:
    res = conn.execute(text("SELECT current_user, current_database(), current_schema();")).fetchone()

print(res)


('patrickpaubandt', 'nf_da_onl_13102025', 'public')


In [5]:
from sqlalchemy import text

with engine.connect() as conn:
    exists = conn.execute(text("""
        SELECT EXISTS(
          SELECT 1
          FROM information_schema.schemata
          WHERE schema_name = :schema
        );
    """), {"schema": PG_SCHEMA}).scalar()

print("Schema exists?", exists)


Schema exists? True


In [6]:
with engine.begin() as conn:
    conn.execute(text(f"CREATE TABLE IF NOT EXISTS {PG_SCHEMA}.perm_test (id INT);"))
    conn.execute(text(f"DROP TABLE {PG_SCHEMA}.perm_test;"))

print("CREATE TABLE test ok in schema:", PG_SCHEMA)


CREATE TABLE test ok in schema: s_patrickpaubandt
